In [ ]:
from PIL import Image, ImageSequence
import numpy as np
import cv2
import tifffile as tiff


def read_imgs(file_name = 'VTA-GC-1T1_G.tiff'):
    imgs = []
# 	img_seq = Image.open(file_name)
    img_seq = tiff.imread(file_name)
    
    for i, img in enumerate(img_seq):
        imgs.append(np.array(img))
    imgs = np.array(imgs) / 2 ** 16 # normalize to [0, 1]
    return imgs


def get_mask(img_shape, boxes, blurness = 0):
	# The larger the blurness value, the stronger the blurness, 0 means no blur, 1 is strongest
	mask = np.zeros(img_shape, dtype=np.float32)
	for box in boxes:
		topleft_x = box[0]
		topleft_y = box[1]
		botright_x = box[2]
		botright_y = box[3]
		box_width = botright_x - topleft_x
		box_height = botright_y - topleft_y
		assert box_width >= 0, "box width should be greater than 0"
		assert box_height >= 0, "box height should be greater than 0"
		assert topleft_x >=0 and topleft_x <= img_shape[1], "topleft_x should be inside the image"
		assert topleft_y >=0 and topleft_y <= img_shape[0], "topleft_y should be inside the image"
		assert botright_x >=0 and botright_x <= img_shape[1], "botright_x should be inside the image"
		assert botright_y >=0 and botright_y <= img_shape[0], "botright_y should be inside the image"
		mask[topleft_y:botright_y, topleft_x:botright_x] = np.ones([box_height, box_width], dtype=np.float32)
	mask = 1 - mask
	if blurness > 0:
		blur_kernel_height = int(blurness * img_shape[0])
		blur_kernel_width = int(blurness * img_shape[1])
		mask = cv2.blur(mask, (blur_kernel_height, blur_kernel_width))
	return mask


def mask_img(img, boxes, mask_blurness=0.1, img_blurness=0.1, mode=0):
	# mask_blurness controls the softness of the mask border
	# img_blurness is only used when mode = 1
	# boxes is a list of box, each box has 4 dims tl_x, tl_y, br_x, br_y
	img_shape = img.shape
	mask = get_mask(img_shape, boxes, mask_blurness)
	if mode == 0:
		# mask out the box area directly,]
		masked_img = img * mask
	elif mode == 1:
    # blur the img, then replace the box area with blured img
		blur_kernel_height = int(img_blurness * img_shape[0])
		blur_kernel_width = int(img_blurness * img_shape[1])
		blured_img = cv2.blur(img, (blur_kernel_height, blur_kernel_width))
		masked_img = img * mask + blured_img * (1 - mask)
	return masked_img





In [ ]:
filename = 'D:/PhD/Microscope/imaging_data_20_09/DA_terminal/VTA-GC-1/processed/2020-10-07/tiff_chained_all.tiff'
save_name = filename[0:-21]+'masked_tiff_chained_all.tiff'
save_name

In [ ]:

imgs = read_imgs(file_name = filename) 
img_shape = [imgs.shape[1], imgs.shape[2]]
box1,box2,box3 = [159,45,199,86],[180,102,197,135],[78,42,103,69]
boxes = [box1,box2,box3]
Masked = []
for img in imgs:
    masked_img = mask_img(img, boxes, 0.1, 0.1, 1)
#     cv2.imshow('masked_img', masked_img)
#     cv2.waitKey()
    Masked.append(np.array(masked_img)*2**16)
tiff.imsave(save_name, np.array(Masked).astype('uint16'),bigtiff=True)

In [ ]:
Masked

In [ ]:
# tiff.imsave(save_name, np.asarray(Masked).astype('uint16'),bigtiff=True)
tiff.imsave(save_name, np.array(Masked).astype('uint16'),bigtiff=True)

In [ ]:
img_seq = tiff.imread(filename)


In [ ]:
img_seq.dtype